# SAP Cloud ALM Project and Task, Processes API Demo
## Copying a Template Project from One Tenant to a Target Project in a Second Tenant

This notebook contains examples of SAP Business Hub API calls for SAP Cloud ALM for Implementation. The code cells below show how you can use APIs published in the SAP API Business Hub to copy a template project in one tenant to target projects in one or more other tenant. This is useful if you wish to distribute a standard, pre-filled project to new projects in your own tenant, or to other client tenants.

The API information and specification is available here:

* https://api.sap.com/package/SAPCloudALM/rest - SAP Cloud ALM
* https://api.sap.com/api/CALM_PJM/overview - SAP Cloud ALM Projects
* https://api.sap.com/api/CALM_TKM/overview - SAP Cloud ALM Tasks
* https://api.sap.com/api/CALM_PM/overview - SAP Cloud ALM Process Scopes

Please note the license and other terms and conditions contained in this notebook's repository: https://github.com/SAP-samples/cloud-alm-api-examples

## Python Dependencies Required

In order to run the samples in this notebook, install the following dependencies:

* Jupyter integration in Visual Studio Code: https://code.visualstudio.com/docs/python/jupyter-support 
* Python 3, a recent version, is Required. Python 3.8 was used here.
* Requests - for handling HTTP GET/POST/PATCH/DELETE Requests - https://requests.readthedocs.io/en/latest/user/install/#install
* Requests-OAuthlib - for authentication with requests - https://requests-oauthlib.readthedocs.io/en/latest/index.html#installation
* Pandas - Python data analysis - https://pandas.pydata.org/docs/getting_started/install.html

## Setting Up Service Instances

In the Business Technology Platform (BTC) Cockpit, you will have to set up service instances for the APIs. During this process, the system will prompt you to upload JSON data. This data contains the authorization scopes required by API callers. As always, take particular care when assigning authorization scopes, and use only the minimum necessary.


Example JSON with just read authorization for template project's tenant:
```json
{
  "xs-security": {
    "xsappname": "<service instance name>",
    "authorities": [
      "$XSMASTERAPPNAME.calm-api.projects.read",
      "$XSMASTERAPPNAME.calm-api.tasks.read",
      "$XSMASTERAPPNAME.calm-api.processmanagement.read",
      "$XSMASTERAPPNAME.calm-api.processauthoring.read"
    ]
  }
}
```

For target projects, allow read and write operations:

```json
{
  "xs-security": {
    "xsappname": "<service instance name>",
    "authorities": [
      "$XSMASTERAPPNAME.calm-api.projects.read",
      "$XSMASTERAPPNAME.calm-api.projects.write",
      "$XSMASTERAPPNAME.calm-api.tasks.write", 
      "$XSMASTERAPPNAME.calm-api.tasks.read",
      "$XSMASTERAPPNAME.calm-api.processmanagement.read",
      "$XSMASTERAPPNAME.calm-api.processmanagement.write",
      "$XSMASTERAPPNAME.calm-api.processmanagement.delete"   
    ]
  }
}
```


---

## Authentication information

You must create a python module file called `apidata.py` and put the information specific to your tenant there. This inclduds

* OAuth2 client ID and secret
* Token url
* Base URL for API calls

Get client ID and secret variables from an external module: this information is senstive.

These items can be retrieved from the BTP Cockpit 

### Format of module apidata.py for import

In this example, you must provide authentication data for two tenants.

1. The _template_ tenant. You read data from this tenant.
2. the _target_ tenant. You copy the data into this tenant.

#### The Template Tenant

```python
template_client_id = r'get your client ID from BTP Cockpit'
template_client_secret = r'get your client secret from BTP Cockpit'
template_token_url = 'your token url'
template_base_url = 'your base url'
```

#### The Target Tenant

```python
target_client_id = r'get your client ID from BTP Cockpit'
target_client_secret = r'get your client secret from BTP Cockpit'
target_token_url = 'your token url'
target_base_url = 'your base url'
```

If you want to copy to multiple target tenants, you could consider creating a collection of target information and iterating through each one.

In [ ]:
# Imports needed for the code below

import requests
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient

import pandas as pd

import apidata as ad # Local file containing authentication credentials

### Get Token for Authentication in Template Tenant

Call OAuth token API with credential information. Add the resulting header to all requests.

See Requests-OAuthlib documentation for Backend Application Flow:

* https://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html#backend-application-flow

In [ ]:
# Read connection information from include:

client_id = ad.template_client_id
client_secret = ad.template_client_secret
token_url = ad.template_token_url
base_url = ad.template_base_url

In [ ]:
# Make call to authenticate, authorize:

client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url=token_url, client_id=client_id,
        client_secret=client_secret)

hed = {'Authorization': 'Bearer ' + token['access_token']}        

### Get Authentication Data for Target Tenant

Perform the same as above, but for the target.


In [ ]:
target_client_id = ad.target_client_id
target_client_secret = ad.target_client_secret
target_token_url = ad.target_token_url
target_base_url = ad.target_base_url

In [ ]:
target_client = BackendApplicationClient(client_id=target_client_id)
target_oauth = OAuth2Session(client=target_client)
target_token = target_oauth.fetch_token(token_url=target_token_url, client_id=target_client_id,
        client_secret=target_client_secret)

target_hed = {'Authorization': 'Bearer ' + target_token['access_token']}        


---

## Perform GET Request to Retrieve List of all Projects from Template Tenant

Expected response: "200 OK"

We place the project list into a Pandas data frame for analysis - this is not necessary, but convenient.

In [ ]:
response = requests.get(base_url + '/api/calm-projects/v1/projects', headers=hed)

print(response.status_code, response.reason)

source_projects = response.json()

df = pd.json_normalize(source_projects)

df

### Set template project name

Change this to the name of the template project which you want to copy.

In [ ]:
# Change to the name of a project you would like to use:

template_project_name = 'Demo Template Project'

## Get ID of a Selected Project

Read ID of template project from dataframe. This will fail if the project does not exist.

In [ ]:
stPrjID = df.loc[df['name'] == template_project_name]
template_project_id = stPrjID['id'].values[0]
print('Project:', stPrjID['name'].values[0], 'ID:', template_project_id)


---

## Read Task list from template project

Use GET request from tasks API with selected project ID as input parameter.

Expected response "200 OK"

In [ ]:
taskURL = base_url + '/api/calm-tasks/v1/tasks?projectId=' + template_project_id

response = requests.get(taskURL, headers=hed)

print(response.status_code, response.reason)
df_template_tasks = pd.json_normalize(response.json())

print("Number of task items in template project:", len(df_template_tasks))

df_template_tasks.head()

## Read Scopes from Template Project

This example uses the SAP Cloud ALM Process Scopes API to get the list of scopes in the template target.

See the API reference here: <https://api.sap.com/api/CALM_PM/resource/Scope>

In [ ]:
response = requests.get(base_url + '/api/calm-processmanagement/v1/scopes/?projectId=' + template_project_id, headers=hed)

print(response.status_code, response.reason)

projectScopes = response.json()

df_scopes = pd.DataFrame.from_dict(projectScopes['value'])

df_scopes

## New - Solution Process Assignments

This is a new enhancement to the tasks API, released 2023-04-19. It allows you to get and set assignments of tasks/user stories/requirements/... to Solution Processes.

API reference: <https://api.sap.com/api/CALM_TKM/resource/Solution_Processes>

### Get all tasks with solution process assignments

This example uses the project ID as a filter. It returns all items (tasks, etc.) in the project with assignments to Solution Processes.


In [ ]:
response = requests.get(base_url + '/api/calm-tasks/v1/tasks/solutionProcessAssignments?projectId=' + template_project_id , headers=hed)

print(response.status_code, response.reason)

dfTasksWithProcesses = pd.DataFrame.from_dict(response.json())
dfTasksWithProcesses

---
## Work on Target Tenant

#### Start with GET Request to Retrieve List of all Projects from Target Tenant

Expected response: "200 OK"

In [ ]:
target_response = requests.get(target_base_url + '/api/calm-projects/v1/projects', headers=target_hed)

print(target_response.status_code, target_response.reason)

target_projects = target_response.json()

print("Number of projects in target tenant:", len(target_projects))

#### Name of Target Project to Create/Update

If you use this in your own system, change the the project name below to the one where you want to copy everything from the template project.


In [ ]:
target_project_name = 'Demo Target Project'
target_project_id = ''

In [ ]:
for project in target_projects:
    if project['name'] == target_project_name:
        target_project_id = project['id']
        print('Project exists: ', target_project_name, target_project_id)
        break
else:
    target_project_id = ''
    print('Project does not exist')

#### Create New Project if it Does not Exist

In [ ]:
if target_project_id == '':

    newProj = {
        "name": target_project_name
    }

    response = requests.post(target_base_url + '/api/calm-projects/v1/projects', headers=target_hed, json=newProj)

    print(response.status_code, response.reason)

    target_project_id = response.json()['id']
    print("New project ID:", target_project_id)

# Refresh the project list:

target_response = requests.get(target_base_url + '/api/calm-projects/v1/projects', headers=target_hed)

print(target_response.status_code, target_response.reason)

### Create Scopes in Target Project

If the scopes already exist, the system responds with `400 Bad Request`.

In [ ]:
df_scopes['targetProjectId'] = target_project_id # Set the ID of the target project in the scopes dataframe

for key, row in df_scopes.iterrows():
    print("Creating scope:", row['name'])

    target_scope = {
        "projectId": target_project_id,
        "name": row['name'],
        "description": row['description']
    }

    response = requests.post(target_base_url + '/api/calm-processmanagement/v1/scopes', headers=target_hed, json=target_scope)

    print(response.status_code, response.reason)

    targetScopeResponse = response.json()


### Read Scope List from Target Project

Read the scope list to get the new IDs from the target project.

In [ ]:
response = requests.get(target_base_url + '/api/calm-processmanagement/v1/scopes/?projectId=' + target_project_id, headers=target_hed)

print(response.status_code, response.reason)


targetProjectScopes = response.json()['value']
targetProjectScopes

#### Map Scope IDs from Template and Target Projects

The code below loops through the scopes in the target project, and writes the IDs (`targetId`) in the corresponding row of the scopes dataframe.

Now the dataframe will have scope and project IDs of both the template and and target projects.

In [ ]:
df_scopes['targetId'] = ''

for scope in targetProjectScopes:
    df_scopes.loc[df_scopes['name'] == scope['name'], 'targetId'] = scope['id']
    
df_scopes

### Add target scope IDs to process assignments

Use a dataframe merge to combine the new scope IDs in the target project with the solution process assignment to tasks.

In [ ]:
df_old_new_processes = pd.merge(dfTasksWithProcesses, df_scopes[['id','targetId']], left_on='scopeId', right_on='id')
df_old_new_processes

### Activate Solution Scenarios and Scope Processes in Target Scopes

There are much more efficient ways of doing this. In particular, adding multiple solution scenario versions to an array would reduce the number of calls.

The code below does the following:

1. Activates the required Solution Scenarios in the respective Scopes.
2. Looks up the needed Solution Process Version ID from the above Solution Scenario.
3. Sets the corresponding Solution Process to in scope.

In [ ]:
target_hed_patch = target_hed.copy()
target_hed_patch.update({'Content-Type': 'application/merge-patch+json'}) # Watch out for this: an explicit content type is required for patch operations

target_hed_post = target_hed.copy()
target_hed_post.update({'Content-Type': 'application/json'}) 


for key, row in df_old_new_processes.iterrows():
    print('Activating Solution Scenario', row['targetId'])
    targetScopeId = row['targetId']
    targetSolutionScenarioVersionId = row['solutionScenarioVersionId']
    targetSolutionProcessId = row['solutionProcessId']


    # 1. Activate the Solution Scenario in the scope
    target_solutionScenarioJson = {"value": [{"id": targetSolutionScenarioVersionId}]}

    response = requests.post(target_base_url + '/api/calm-processmanagement/v1/scopes/' + targetScopeId + '/solutionScenarioVersions' , headers=target_hed_post, json=target_solutionScenarioJson)

    print(response.status_code, response.reason)

    # 2. Look up the Solution Process Version ID

    response = requests.get(target_base_url + '/api/calm-processmanagement/v1/solutionProcesses?solutionScenarioVersionId=' + targetSolutionScenarioVersionId + '&projectId=' + target_project_id + '&scopeId=' + targetScopeId + '&solutionProcessId=' + targetSolutionProcessId, headers=target_hed)

    print(response.status_code, response.reason)

    response.json()

    targetSolutionProcessResponse = response.json()

    targetSolutionProcessVersionId = targetSolutionProcessResponse['value'][0]['solutionProcessVersionId']

    print('Version ID for process', targetSolutionProcessId, targetSolutionProcessVersionId)


    # 3. Set the Solution Process to in scope:

    print('Setting Solution Process to in scope', targetSolutionProcessId)

    solutionProcessScopeAssigmentJson = {'value': [{'scopeId': targetScopeId,
        'solutionScenarioVersionId': targetSolutionScenarioVersionId,
        'solutionProcessVersionId': targetSolutionProcessVersionId,
        'statusId': 'DESIGN',
        'isScoped': True}]}


    response = requests.patch(target_base_url + '/api/calm-processmanagement/v1/solutionProcesses/scopeAssignments', headers=target_hed_patch, json=solutionProcessScopeAssigmentJson)

    print(response.status_code, response.reason)


## Create tasks in target project

This is the first pass which just creates the tasks, but does not create any sub-tasks. The tasks have to be in place before any sub-tasks may be assigned to them.

We use the `externalId` field to store the original ID of the template task, so we can copy subtasks and process assignments from the template task to the new task.

In [ ]:
taskCreateURL = target_base_url + '/api/calm-tasks/v1/tasks'

for key, row in df_template_tasks.iterrows():

    # Skip the roadmap tasks, and skip subtasks for now:
    if row['type'] != 'CALMTMPL' and row['type'] != 'CALMST':
        print(row['title'])

        template_taskURL = base_url + '/api/calm-tasks/v1/tasks/' + row['id']

        response = requests.get(template_taskURL, headers=hed)

        templateTaskDetails = response.json()

        templateTaskDetails['projectId'] = target_project_id # use the new project id
        templateTaskDetails['externalId'] = row['id'] # set the original task id as the external id

        templateScopeId = templateTaskDetails['scopeId']

        if templateScopeId == None:
            templateTaskDetails.pop('scopeId', None) # remove the scope id if it is null
        else:

            targetScopeId = df_scopes['targetId'].loc[ df_scopes['id'] == templateScopeId ] 

            templateTaskDetails['scopeId'] = targetScopeId.values[0] # Otherwise set the new scope ID


        # Remove attributes which are invalid, or get set automatically by the system:
        templateTaskDetails.pop('id', None)
        templateTaskDetails.pop('assigneeId', None)
        templateTaskDetails.pop('createdTimestamp', None)
        templateTaskDetails.pop('lastChangedTimestamp', None)
  

        response = requests.post(taskCreateURL, headers=target_hed, json=templateTaskDetails)

        print(response.status_code, response.reason)

        newTaskID = response.json()['id']
        print("New task ID:", newTaskID)

        #break

## Re-read target task list

May need to refresh buffers

In [ ]:
taskURL = target_base_url + '/api/calm-tasks/v1/tasks?projectId=' + target_project_id

response = requests.get(taskURL, headers=target_hed)

print(response.status_code, response.reason)

df_target_tasks = pd.json_normalize(response.json())

print("Number of tasks in target project:", len(df_target_tasks))

df_target_tasks.loc[df_target_tasks['externalId'].notnull()].head() # Display the first tasks with external IDs

### Merge the template tasks with process assignments with the target task IDs

In [ ]:
df_copied_task_mapping = df_target_tasks.loc[df_target_tasks['externalId'].notnull(), ('id', 'externalId')]
dfTasksWithProcesses = pd.merge(dfTasksWithProcesses, df_copied_task_mapping, left_on='taskId', right_on='externalId')
dfTasksWithProcesses.rename(columns={"id": "targetTaskId"}, inplace=True)
dfTasksWithProcesses

### Merge the target scope IDs with the task list

In [ ]:
dfTasksWithProcesses = pd.merge(dfTasksWithProcesses, df_scopes[["id", "targetId"]], left_on="scopeId", right_on="id" ).rename(columns={"targetId": "targetScopeId"})
dfTasksWithProcesses

### Perform Solution Process Assignments

The following code loops through all the tasks with process assignments and creates the assignments in the target project:

In [ ]:
for key, row in dfTasksWithProcesses.iterrows():
    print("Updating task with ID", row["targetTaskId"])

    solutionProcessToTaskJson = [
        {
            "taskId": row["targetTaskId"],
            "scopeId": row["targetScopeId"],
            "solutionScenarioVersionId": row["solutionScenarioVersionId"],
            "solutionProcessId": row["solutionProcessId"]
        }
    ]
    response = requests.post(target_base_url + '/api/calm-tasks/v1/tasks/solutionProcessAssignments' , headers=target_hed, json=solutionProcessToTaskJson)

    print(response.status_code, response.reason)



### Add subtasks, looking up new parent task

Use the external ID of the target task to match the original template parent

In [ ]:
for key, row in df_template_tasks.iterrows():

    # Just get sub-tasks:
    if row['type'] == 'CALMST':


        template_taskURL = base_url + '/api/calm-tasks/v1/tasks/' + row['id']

        response = requests.get(template_taskURL, headers=hed)

        templateTaskDetails = response.json()

        print(templateTaskDetails['title'])

        target_parent_task = df_target_tasks.loc[df_target_tasks['externalId'] == templateTaskDetails['parentId']]

        templateTaskDetails['parentId'] = target_parent_task['id'].values[0]

            
        templateTaskDetails['projectId'] = target_project_id
        templateTaskDetails['externalId'] = row['id']

        templateScopeId = templateTaskDetails['scopeId']

        if templateScopeId == None:
            templateTaskDetails.pop('scopeId', None)
        else:
            targetScopeId = df_scopes['targetId'].loc[ df_scopes['id'] == templateScopeId ] 
            print('Target Scope ID', targetScopeId.values[0])
            templateTaskDetails['scopeId'] = targetScopeId.values[0]


        templateTaskDetails.pop('id', None)
        templateTaskDetails.pop('assigneeId', None)
        #
        templateTaskDetails.pop('createdTimestamp', None)
        templateTaskDetails.pop('lastChangedTimestamp', None)
        #templateTaskDetails.pop('parentId', None)

        response = requests.post(taskCreateURL, headers=target_hed, json=templateTaskDetails)

        print(response.status_code, response.reason)

        newTaskID = response.json()['id']
        print("New task ID:", newTaskID)

### Cleanup: Delete Tasks 

taskURL = target_base_url + '/api/calm-tasks/v1/tasks?projectId=' + target_project_id

response = requests.get(taskURL, headers=target_hed)

print(response.status_code, response.reason)

for task in response.json():
    if task['type'] != 'CALMTMPL':
        print(task['id'], task['type'], task['title'])

        taskURL = target_base_url + '/api/calm-tasks/v1/tasks/' + task['id']

        response = requests.delete(taskURL, headers=target_hed)

        print(response.status_code, response.reason)

        #break

## Delete Template Tasks in Target Project

taskURL = target_base_url + '/api/calm-tasks/v1/tasks?projectId=' + target_project_id

response = requests.get(taskURL, headers=target_hed)

print(response.status_code, response.reason)

for task in response.json():
    if task['type'] == 'CALMTMPL':
        print(task['id'], task['type'], task['title'])

        taskURL = target_base_url + '/api/calm-tasks/v1/tasks/' + task['id']

        response = requests.delete(taskURL, headers=target_hed)

        print(response.status_code, response.reason)

        #break

End of Target Project Create

---